

# Exploring Toronto Trending Venues

### - First we import all the required libraries -

In [1]:
import pandas as pd
import requests as req
from bs4 import BeautifulSoup as bs
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
from sklearn.cluster import KMeans
!pip install geopy
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
! pip install folium==0.5.0
import folium # plotting library

### - Now we proceed to create the same dataframe as part 2 -

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = req.get(link).text
wiki = bs(data, "html")

In [3]:
table = wiki.find('table')

In [4]:
cn = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = cn)

In [5]:
for tr in table.find_all('tr'):
    row_data=[]
    for td in tr.find_all('td'):
        row_data.append(td.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [6]:
df=df[df['Borough']!='Not assigned']
i = 0
for i in range(0,df.shape[0]):
    if df.iloc[i][2] == 'Not assigned':
        df.iloc[i][2] = df.iloc[i][1]
        i = i+1

In [7]:
df = df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [8]:
df = df.dropna()
df = df[(df.Postalcode != 'Not assigned' )]

In [9]:
print(df.shape)

(103, 3)


In [10]:
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/

--2020-12-09 06:55:46--  http://cocl.us/Geospatial_data/
Resolving cocl.us (cocl.us)... 169.63.96.176, 169.63.96.194
Connecting to cocl.us (cocl.us)|169.63.96.176|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/ [following]
--2020-12-09 06:55:47--  https://cocl.us/Geospatial_data/
Connecting to cocl.us (cocl.us)|169.63.96.176|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-12-09 06:55:48--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.29.197
Connecting to ibm.box.com (ibm.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-12-09 06:55:48--  https://ibm.box.com/public/static/9afzr

In [11]:
dfgeo = pd.read_csv('GeoCord.csv')
dfgeo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [12]:
df = pd.merge(df,dfgeo[["Postal Code","Latitude","Longitude"]],left_on="Postalcode", right_on="Postal Code", how = "left")
df.drop(columns = "Postal Code")

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


### Now we'll explore the venues using Foursquare

In [13]:
CLIENT_ID = 'PZJ0BKURUBDEFSBLRRJW2DD0TZR3BZKYTDYBANFI0MPPHBJB' 
CLIENT_SECRET = 'ICKZENC5PODMHCQUPSPQ01E22TKXC01FFQRV4IMI5AZXWVK3' 
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PZJ0BKURUBDEFSBLRRJW2DD0TZR3BZKYTDYBANFI0MPPHBJB
CLIENT_SECRET:ICKZENC5PODMHCQUPSPQ01E22TKXC01FFQRV4IMI5AZXWVK3


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [15]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### - We'll focus on Weston Neighborhoor and nearby neighborhoods -

In [16]:
westondf = df[df['Neighborhood'].str.contains("Weston")].reset_index(drop=True)
westondf.head()

,Postalcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M9N,York,Weston,M9N,43.706876,-79.518188


In [17]:
latitude = westondf.loc[0]["Latitude"]
longitude = westondf.loc[0]["Longitude"]
latitude
longitude

-79.51818840000001

In [18]:
df

,Postalcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
...,...,...,...,...,...,...
98,M9N,York,Weston,M9N,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,M9P,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",M9R,43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",M9V,43.739416,-79.588437


In [19]:
radius = 1000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=PZJ0BKURUBDEFSBLRRJW2DD0TZR3BZKYTDYBANFI0MPPHBJB&client_secret=ICKZENC5PODMHCQUPSPQ01E22TKXC01FFQRV4IMI5AZXWVK3&v=20180604&ll=43.706876,-79.51818840000001&radius=1000&limit=100'

In [20]:
results = requests.get(url).json()

In [21]:
venues = results['response']['groups'][0]['items']

In [22]:
nearby_venues = json_normalize(venues)
nearby_venues

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,...,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.postalCode,venue.location.neighborhood,venue.location.crossStreet
0,e-0-4c670e987abde21e0b6d6568-0,0,"[{'summary': 'This spot is popular', 'type': '...",4c670e987abde21e0b6d6568,P&M Restaurant,1972 Weston Road,43.701108,-79.518982,"[{'label': 'display', 'lat': 43.70110845129989...",645,...,Toronto,ON,Canada,"[1972 Weston Road, Toronto ON, Canada]","[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",0,[],NaN,NaN,NaN
1,e-0-562af766498e1f9621e63af1-1,0,"[{'summary': 'This spot is popular', 'type': '...",562af766498e1f9621e63af1,UP Express - Weston Station,1865 Weston Rd,43.700256,-79.513660,"[{'label': 'display', 'lat': 43.700256, 'lng':...",822,...,Toronto,ON,Canada,"[1865 Weston Rd, Toronto ON M9N 1V9, Canada]","[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",0,[],M9N 1V9,Weston,NaN
2,e-0-4c3b9d8d5810a593b1f8ba3c-2,0,"[{'summary': 'This spot is popular', 'type': '...",4c3b9d8d5810a593b1f8ba3c,Shoppers Drug Mart,1995 Weston Rd,43.701264,-79.519705,"[{'label': 'display', 'lat': 43.701264, 'lng':...",636,...,Toronto,ON,Canada,"[1995 Weston Rd (at Lawrence Ave. W.), Toronto...","[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",0,[],M9N 1X2,NaN,at Lawrence Ave. W.
3,e-0-4cddd9efdb125481cbf32cce-3,0,"[{'summary': 'This spot is popular', 'type': '...",4cddd9efdb125481cbf32cce,Weston Lions Arena,2125 Lawrence Ave West,43.698707,-79.518548,"[{'label': 'display', 'lat': 43.69870670108964...",909,...,Toronto,ON,Canada,"[2125 Lawrence Ave West, Toronto ON, Canada]","[{'id': '4bf58dd8d48988d168941735', 'name': 'S...",0,[],NaN,NaN,NaN
4,e-0-58c2bcd3bf1a6d6b319c1499-4,0,"[{'summary': 'This spot is popular', 'type': '...",58c2bcd3bf1a6d6b319c1499,Tim Hortons,2013 Lawrence Ave West,43.700815,-79.511638,"[{'label': 'display', 'lat': 43.7008152218159,...",856,...,North York,ON,Canada,"[2013 Lawrence Ave West, North York ON M9N 0A3...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],M9N 0A3,NaN,NaN
5,e-0-4cc2280b06c254812dfd8747-5,0,"[{'summary': 'This spot is popular', 'type': '...",4cc2280b06c254812dfd8747,Pizza Pizza,1937 Weston Rd.,43.700589,-79.517499,"[{'label': 'display', 'lat': 43.70058873986015...",702,...,Toronto,ON,Canada,"[1937 Weston Rd., Toronto ON M9N 1W7, Canada]","[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[],M9N 1W7,NaN,NaN
6,e-0-4e52ee7881dc6e249dd940aa-6,0,"[{'summary': 'This spot is popular', 'type': '...",4e52ee7881dc6e249dd940aa,Popeyes Louisiana Kitchen,1975 Weston Rd.,43.701045,-79.518921,"[{'label': 'display', 'lat': 43.70104530536844...",651,...,Toronto,ON,Canada,"[1975 Weston Rd. (at Little Ave.), Toronto ON ...","[{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'F...",0,[],M9N 1W8,NaN,at Little Ave.
7,e-0-4b96d5f7f964a520cee734e3-7,0,"[{'summary': 'This spot is popular', 'type': '...",4b96d5f7f964a520cee734e3,Tim Hortons,2013 Lawrence Ave W,43.700910,-79.511692,"[{'label': 'display', 'lat': 43.70090969892793...",845,...,Toronto,ON,Canada,"[2013 Lawrence Ave W (Ralph St.), Toronto ON M...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],M9N 0A3,NaN,Ralph St.
8,e-0-4e3c0a5cd164b52911992aca-8,0,"[{'summary': 'This spot is popular', 'type': '...",4e3c0a5cd164b52911992aca,Grattan Park,NaN,43.706222,-79.521705,"[{'label': 'display', 'lat': 43.70622161954314...",292,...,NaN,Ontario,Canada,"[Ontario, Canada]","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],NaN,NaN,NaN
9,e-0-4b9a409ef964a52079a735e3-9,0,"[{'summary': 'This spot is popular', 'type': '...",4b9a409ef964a52079a735e3,Weston GO Station,39 John St,43.700212,-79.513345,"[{'label': 'display', 'lat': 43.70021244961973...",837,...,Etobicoke,ON,Canada,"[39 John St, Etobicoke ON, Canada]","[{'id': 

In [23]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,P&M Restaurant,Diner,43.701108,-79.518982
1,UP Express - Weston Station,Train Station,43.700256,-79.513660
2,Shoppers Drug Mart,Pharmacy,43.701264,-79.519705
3,Weston Lions Arena,Skating Rink,43.698707,-79.518548
4,Tim Hortons,Coffee Shop,43.700815,-79.511638


In [24]:
nearby_venues

,name,categories,lat,lng
0,P&M Restaurant,Diner,43.701108,-79.518982
1,UP Express - Weston Station,Train Station,43.700256,-79.513660
2,Shoppers Drug Mart,Pharmacy,43.701264,-79.519705
3,Weston Lions Arena,Skating Rink,43.698707,-79.518548
4,Tim Hortons,Coffee Shop,43.700815,-79.511638
5,Pizza Pizza,Pizza Place,43.700589,-79.517499
6,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.701045,-79.518921
7,Tim Hortons,Coffee Shop,43.700910,-79.511692
8,Grattan Park,Park,43.706222,-79.521705
9,Weston GO Station,Train Station,43.700212,-79.513345


In [25]:
df.head()

,Postalcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [26]:
venues = getNearbyVenues(names=df['Neighborhood'],latitudes=df['Latitude'],longitudes=df['Longitude'])

In [27]:
venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
...,...,...,...,...,...,...,...
2136,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,LCBO,43.741508,-79.584501,Liquor Store
2137,"Northwest, West Humber - Clairville",43.706748,-79.594054,Economy Rent A Car,43.708471,-79.589943,Rental Car Location
2138,"Northwest, West Humber - Clairville",43.706748,-79.594054,Logistics Distribution,43.707554,-79.589252,Bar
2139,"Northwest, West Humber - Clairville",43.706748,-79.594054,Saand Rexdale,43.705072,-79.598725,Drugstore


### - We'll analyze the each neighborhood -

In [28]:
VC = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

VC['Neighborhood'] = venues['Neighborhood'] 

VC.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
groupnh = VC.groupby('Neighborhood').mean().reset_index()
groupnh

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
num_top_venues = 5

for hood in groupnh['Neighborhood']:
    print("----"+hood+"----")
    temp = groupnh[groupnh['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1             Clothing Store   0.2
2             Breakfast Spot   0.2
3                     Lounge   0.2
4               Skating Rink   0.2


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1             Gym  0.14
2  Sandwich Place  0.14
3             Pub  0.14
4     Coffee Shop  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Coffee Shop  0.10
1                       Bank  0.10
2                   Pharmacy  0.05
3              Shopping Mall  0.05
4  Middle Eastern Restaurant  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3                 Café  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0              Coffee Shop  0.09
1           Sandw

                  venue  freq
0           Social Club  0.06
1                   Gym  0.06
2         Burrito Place  0.06
3  Fast Food Restaurant  0.06
4        Sandwich Place  0.06


----Moore Park, Summerhill East----
                             venue  freq
0                       Playground   0.5
1                            Trail   0.5
2                Accessories Store   0.0
3               Mexican Restaurant   0.0
4  Molecular Gastronomy Restaurant   0.0


----New Toronto, Mimico South, Humber Bay Shores----
                 venue  freq
0                 Café  0.14
1          Coffee Shop  0.14
2          Pizza Place  0.07
3  American Restaurant  0.07
4           Restaurant  0.07


----North Park, Maple Leaf Park, Upwood Park----
                        venue  freq
0                      Bakery  0.33
1  Construction & Landscaping  0.33
2                        Park  0.33
3           Accessories Store  0.00
4          Mexican Restaurant  0.00


----North Toronto West,  Lawrence Park

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 5
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Neighborhood'] = groupnh['Neighborhood']

for ind in np.arange(groupnh.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(groupnh.iloc[ind, :], num_top_venues)

venues_sorted.shape

(96, 6)

In [33]:
print(venues_sorted)

                                       Neighborhood 1st Most Common Venue  \
0                                         Agincourt                Lounge   
1                            Alderwood, Long Branch           Pizza Place   
2   Bathurst Manor, Wilson Heights, Downsview North                  Bank   
3                                   Bayview Village                  Café   
4                 Bedford Park, Lawrence Manor East        Sandwich Place   
..                                              ...                   ...   
91                      Willowdale, Willowdale West           Pizza Place   
92                                           Woburn           Coffee Shop   
93                                 Woodbine Heights          Skating Rink   
94                                  York Mills West                  Park   
95                         York Mills, Silver Hills   Martial Arts School   

    2nd Most Common Venue      3rd Most Common Venue 4th Most Common Venue 

In [34]:
df.head()

,Postalcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


### - Now we cluster the neighborhoods -

In [35]:
kclusters = 6
groupnh_clustering = groupnh.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(groupnh_clustering)
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [36]:
venues_sorted.insert(0, 'Cluster l', kmeans.labels_)
toronto_merged = df
toronto_merged = toronto_merged.join(venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster l,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353,4.0,Fast Food Restaurant,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497,1.0,Construction & Landscaping,Bar,Yoga Studio,Eastern European Restaurant,Dog Run
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711,1.0,Breakfast Spot,Restaurant,Electronics Store,Rental Car Location,Medical Center
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917,1.0,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,Dumpling Restaurant,Distribution Center
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476,1.0,Caribbean Restaurant,Hakka Restaurant,Bakery,Gas Station,Athletics & Sports


### - To finish this analysis, we need to visualize the clusters in a map -

In [37]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters